In [2]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime

import unidecode

In [163]:
datos_prestador = querydbtopandas(f"""select top 1
       a13.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno
    from  DBA.d_pre_prestador        a13
      JOIN  DBA.d_prof_convertir_especialidad a19  
        ON  (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a13.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
    where  a13.id_pre_prestador in ({prestador})""").values[0]

nombre = datos_prestador[1]

especialidad = datos_prestador[2]

In [3]:
prestaciones_excluidas = pd.read_excel("prestaciones_excluidas.xlsx",usecols = ["prestac_pres_prestacion"], dtype=str )["prestac_pres_prestacion"].tolist()
prestaciones_excluidas = str(prestaciones_excluidas).strip("[").strip("]")

In [4]:
provincias_zonas = pd.read_csv("provincias_zonas.csv")

In [5]:
provincias_zonas.codigo_zona.unique()

array([  1,   2,   3,   4,   5, 100])

In [6]:
provincias_zonas.columns

Index(['codigo_zona', 'zona', 'provincia', 'codigo_provincia '], dtype='object')

In [7]:
provincias_zonas["codigo_provincia "].unique()

array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 17, 18, 19,
       20, 21, 22, 23, 24, 25,  2, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
       61, 62, 63, 64, 65, 66, 68])

In [8]:
# clasificacion incorrecta del nomenclador
# no oncluyen cientro prestadores
zona = "2,3,4,5"
provincia = "1,2"

In [59]:
menores = ["2019-01-01", "2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01", "2020-01-01"]

mayores = ["2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01","2020-01-01", "2020-02-01"]

temporal_prestador = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])

temporal_especialidad = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])

#menor = str(datetime.datetime.today() - datetime.timedelta(days=104))[:10]
#mayor = str(datetime.datetime.today() - datetime.timedelta(days=74))[:10]

In [137]:
prestador = 117056

In [138]:
especialidad = querydbtopandas(f"""select top 1
       a13.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno
from  DBA.d_pre_prestador        a13
      JOIN  DBA.d_prof_convertir_especialidad a19  
        ON  (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a13.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
where  a13.id_pre_prestador in ({prestador})""").values[0][2]
especialidad = unidecode.unidecode(especialidad)

In [140]:
especialidad 

'Nefrologia'

In [141]:
for i,j in zip(menores,mayores):
    df = querydbtopandas(f"""select a11.id_pres_prestacion,
       a16.prestac_pres_prestacion,
       a11.id_pre_prestador,
       a13.desc_pre_nombre,
       a11.id_afi_afiliado ,  
       Q.deno,
       a11.id_tie_dia_consumo,   
       FLOOR(DATEDIFF(year, a14.naci_fecha, a11.id_tie_dia_consumo)) edad_afiliado 

from        DBA.ft_ordenes        a11
       join        DBA.d_pre_prestador        a13
         on        (a11.id_pre_prestador = a13.id_pre_prestador)
       join        DBA.afi_afiliados        a14
         on        (a11.id_afi_afiliado = a14.id_afiliado)
       join        DBA.d_pres_prestacion        a16
         on        (a11.id_pres_prestacion = a16.id_pres_prestacion)     
       join       DBA.d_prof_convertir_especialidad a19  
        on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a11.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
where        (a13.id_pre_tipo in ('E', 'P')            
and a11.id_nom_nomenclador in (5)            
and cast(id_tie_dia_consumo as date) >= date("{i}") and cast(id_tie_dia_consumo as date) < date("{j}")
and a13.id_afi_zona in ({zona})
and a13.id_afi_provincia in ({provincia})
and Q.deno like "%{especialidad}%" 
and RTRIM(a16.prestac_pres_prestacion) not in ({prestaciones_excluidas}))
group by a11.id_pres_prestacion,
       a16.prestac_pres_prestacion,
       a11.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno,
       a11.id_afi_afiliado,
       a11.id_tie_dia_consumo,
       edad_afiliado""")
    
    total_consultas_prestador = df["id_pre_prestador"].value_counts()
    mas_de_30_consultas = total_consultas_prestador[total_consultas_prestador >= 30].index

    df = df[df["id_pre_prestador"].isin(mas_de_30_consultas)]

    reconsultas = df.groupby(["id_pre_prestador", "desc_pre_nombre", "deno", "id_afi_afiliado"])["id_afi_afiliado"].count().rename("CANTIDADORDENES0").reset_index()
    reconsultas["uno"] = 1

    pivot_1 = pd.pivot_table(reconsultas, values='uno', index=['id_pre_prestador', "desc_pre_nombre", "deno"],
                 columns=['CANTIDADORDENES0'], aggfunc=np.sum).fillna(0)

    pivot_1["Más de 4 - Q afiliados"] = pivot_1[pivot_1.columns.tolist()[4:]].sum(axis=1)

    total_consultas_prestador = reconsultas.groupby(["id_pre_prestador",  "desc_pre_nombre", "deno"])["CANTIDADORDENES0"].sum()
    pivot = pd.concat([total_consultas_prestador, pivot_1],axis=1)
    pivot.reset_index(inplace=True)
    if 1 not in pivot.columns:
        pivot[1] = 0
    if 2 not in pivot.columns:
        pivot[2] = 0
    if 3 not in pivot.columns:
        pivot[3] = 0
    if 4 not in pivot.columns:
        pivot[4] = 0
    
    pivot["Consultas - participación %"] =  (pivot[1]/pivot["CANTIDADORDENES0"])
    pivot["2 consultas - participación %"] =  ((pivot[2]*2)/pivot["CANTIDADORDENES0"])
    pivot["3 consultas - participación %"] =  ((pivot[3]*3)/pivot["CANTIDADORDENES0"])
    pivot["4 consultas - participación %"] =  ((pivot[4]*4)/pivot["CANTIDADORDENES0"])
    pivot["Participación %"] = 1 - pivot["Consultas - participación %"]
    pivot["Más de 4 - participación %"] = pivot["Participación %"] - (pivot["2 consultas - participación %"]+pivot["3 consultas - participación %"]+pivot["4 consultas - participación %"])

    pivot.rename(columns= {1: "Consultas - Q afiliados", 2: "2 consultas - Q afiliados", 3: "3 consultas - Q afiliados", 4: "4 consultas - Q afiliados", "desc_pre_nombre": "Razon social",
    "deno": "Especialidad","id_pre_prestador": "Prestador", "edad_afiliado": "Promedio edad de afiliados", "CANTIDADORDENES0": "Total Consultas" },inplace=True)    
    
    tabla = pivot[['Prestador', 'Razon social', 'Especialidad',
              'Consultas - participación %', '2 consultas - participación %',
       '3 consultas - participación %', '4 consultas - participación %',
                'Más de 4 - participación %']]
    
    if len(tabla[tabla["Prestador"] == 218655]) ==0:
        temporal_prestador[i] = 0
        temporal_especialidad[i] = 0
    else:
        especialidad_prestador = tabla[tabla["Prestador"] == 218655].values[0][2]
        prestador = tabla[tabla["Prestador"] == 218655].iloc[:,3:]
        temporal_prestador[i] = prestador.T.values
        temporal_especialidad[i] = tabla[tabla["Especialidad"] == especialidad_prestador].iloc[:,3:].mean(axis=0).values
   

In [144]:
temporal_prestador = temporal_prestador.melt(id_vars="consulta", value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
temporal_prestador["tipo"] = "prestador"

temporal_especialidad = temporal_especialidad.melt(id_vars="consulta",  value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
temporal_especialidad["tipo"] = "especialidad"

In [145]:
final = pd.concat([temporal_especialidad,temporal_prestador])
final = final.round(2)

In [146]:
di = {"Consultas - Q afiliados Médica": "Porcentaje de afiliados con consultas únicas", 
     "2 consultas - Q afiliados": "Porcentaje de afiliados con 2 consultas",
     "3 consultas - Q afiliado": "Porcentaje de afiliados con 3 consultas", 
     "4 consultas - Q afiliados": "Porcentaje de afiliados con 4 consultas",
     "Más de 4 - Q afiliados": "Porcentaje de afiliados con más de 4 consultas"}
final = final.replace({"consulta": di})

In [147]:
final = final.rename(columns={"value": "Participación mensual"})

In [148]:
import plotly.express as px

In [220]:
fig = px.bar(final, x="tipo", y="Participación mensual", barmode="stack",
             facet_col="variable",  color="consulta")

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1][:-3]))


fig.layout.xaxis1.tickangle = -90
fig.layout.xaxis2.tickangle = -90
fig.layout.xaxis3.tickangle = -90
fig.layout.xaxis4.tickangle = -90
fig.layout.xaxis5.tickangle = -90
fig.layout.xaxis5.tickangle = -90
fig.layout.xaxis6.tickangle = -90
fig.layout.xaxis7.tickangle = -90
fig.layout.xaxis8.tickangle = -90
fig.layout.xaxis9.tickangle = -90
fig.layout.xaxis10.tickangle = -90
fig.layout.xaxis11.tickangle = -90
fig.layout.xaxis12.tickangle = -90
fig.layout.xaxis13.tickangle =-90

fig.layout.xaxis1.title = None
fig.layout.xaxis2.title = None
fig.layout.xaxis3.title = None
fig.layout.xaxis4.title = None
fig.layout.xaxis5.title = None
fig.layout.xaxis5.title = None
fig.layout.xaxis6.title = None
fig.layout.xaxis7.title = None
fig.layout.xaxis8.title = None
fig.layout.xaxis9.title = None
fig.layout.xaxis10.title = None
fig.layout.xaxis11.title = None
fig.layout.xaxis12.title = None
fig.layout.xaxis13.title = None


for i in range(0,13):
    fig.data[i]["marker"]["color"] = "#45b6fe"
    
for i in range(13,26):
    fig.data[i]["marker"]["color"] = "#3792cb"
    
for i in range(26,39):
    fig.data[i]["marker"]["color"] = "#296d98"
    
for i in range(39,52):
    fig.data[i]["marker"]["color"] = "#1c4966"
for i in range(52,65):
    fig.data[i]["marker"]["color"] = "black"

fig.layout.legend = {"orientation": "h", "xanchor" :"center", "y":-0.4, "x":0.5}
fig.update_yaxes(range=[-0,1],tickformat = "%")
fig.update_layout(title=dict(text="Prestador", x=0.5))

In [157]:
prestador = 218655

In [159]:
especialidad = querydbtopandas(f"""select top 1
       a13.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno
    from  DBA.d_pre_prestador        a13
      JOIN  DBA.d_prof_convertir_especialidad a19  
        ON  (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a13.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
    where  a13.id_pre_prestador in ({prestador})""").values[0][2]
especialidad = unidecode.unidecode(especialidad)
    

#consultas
menores = ["2019-01-01", "2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01", "2020-01-01"]

mayores = ["2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01","2020-01-01", "2020-02-01"]

temporal_prestador_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])

temporal_especialidad_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])
    
for i,j in zip(menores,mayores):
    df = querydbtopandas(f"""select a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           a11.id_afi_afiliado ,  
           Q.deno,
           a11.id_tie_dia_consumo,   
           FLOOR(DATEDIFF(year, a14.naci_fecha, a11.id_tie_dia_consumo)) edad_afiliado 
    from        DBA.ft_ordenes        a11
           join        DBA.d_pre_prestador        a13
             on        (a11.id_pre_prestador = a13.id_pre_prestador)
           join        DBA.afi_afiliados        a14
             on        (a11.id_afi_afiliado = a14.id_afiliado)
           join        DBA.d_pres_prestacion        a16
             on        (a11.id_pres_prestacion = a16.id_pres_prestacion)     
           join       DBA.d_prof_convertir_especialidad a19  
            on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)
           LEFT JOIN dba.prestad_costo_centros E
            ON a11.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
           LEFT JOIN dba.costo_centros Q
            ON E.centro = Q.centro
    where   (a13.id_pre_tipo in ('E', 'P')            
    and a11.id_nom_nomenclador in (5)            
    and cast(id_tie_dia_consumo as date) >= date("{i}") and cast(id_tie_dia_consumo as date) < date("{j}")
    and a13.id_afi_zona in ({zona})
    and a13.id_afi_provincia in ({provincia})
    and Q.deno like "%{especialidad}%" 
    and RTRIM(a16.prestac_pres_prestacion) not in ({prestaciones_excluidas}))
    group by a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           Q.deno,
           a11.id_afi_afiliado,
           a11.id_tie_dia_consumo,
           edad_afiliado""")

In [172]:
prestador= 218655

In [173]:
def calculo_graficos(prestador):
    
    prestador= int(prestador)
    #especialidad y nombre
    
    datos_prestador = querydbtopandas(f"""select top 1
       a13.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno
    from  DBA.d_pre_prestador        a13
      JOIN  DBA.d_prof_convertir_especialidad a19  
        ON  (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a13.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
    where  a13.id_pre_prestador in ({prestador})""").values[0]
    nombre = datos_prestador[1]
    especialidad = datos_prestador[2]
    especialidad = unidecode.unidecode(especialidad)

    #consultas
    menores = ["2019-01-01", "2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01", "2020-01-01"]

    mayores = ["2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01","2020-01-01", "2020-02-01"]

    temporal_prestador_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])

    temporal_especialidad_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])
    
    for i,j in zip(menores,mayores):
        df = querydbtopandas(f"""select a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           a11.id_afi_afiliado ,  
           Q.deno,
           a11.id_tie_dia_consumo,   
           FLOOR(DATEDIFF(year, a14.naci_fecha, a11.id_tie_dia_consumo)) edad_afiliado 

    from        DBA.ft_ordenes        a11
           join        DBA.d_pre_prestador        a13
             on        (a11.id_pre_prestador = a13.id_pre_prestador)
           join        DBA.afi_afiliados        a14
             on        (a11.id_afi_afiliado = a14.id_afiliado)
           join        DBA.d_pres_prestacion        a16
             on        (a11.id_pres_prestacion = a16.id_pres_prestacion)     
           join       DBA.d_prof_convertir_especialidad a19  
            on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)
           LEFT JOIN dba.prestad_costo_centros E
            ON a11.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
           LEFT JOIN dba.costo_centros Q
            ON E.centro = Q.centro
    where        (a13.id_pre_tipo in ('E', 'P')            
    and a11.id_nom_nomenclador in (5)            
    and cast(id_tie_dia_consumo as date) >= date("{i}") and cast(id_tie_dia_consumo as date) < date("{j}")
    and a13.id_afi_zona in ({zona})
    and a13.id_afi_provincia in ({provincia})
    and Q.deno like "%{especialidad}%" 
    and RTRIM(a16.prestac_pres_prestacion) not in ({prestaciones_excluidas}))
    group by a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           Q.deno,
           a11.id_afi_afiliado,
           a11.id_tie_dia_consumo,
           edad_afiliado""")

        total_consultas_prestador = df["id_pre_prestador"].value_counts()
        mas_de_30_consultas = total_consultas_prestador[total_consultas_prestador >= 30].index

        df = df[df["id_pre_prestador"].isin(mas_de_30_consultas)]

        reconsultas = df.groupby(["id_pre_prestador", "desc_pre_nombre", "deno", "id_afi_afiliado"])["id_afi_afiliado"].count().rename("CANTIDADORDENES0").reset_index()
        reconsultas["uno"] = 1

        pivot_1 = pd.pivot_table(reconsultas, values='uno', index=['id_pre_prestador', "desc_pre_nombre", "deno"],
                     columns=['CANTIDADORDENES0'], aggfunc=np.sum).fillna(0)

        pivot_1["Más de 4 - Q afiliados"] = pivot_1[pivot_1.columns.tolist()[4:]].sum(axis=1)

        total_consultas_prestador = reconsultas.groupby(["id_pre_prestador",  "desc_pre_nombre", "deno"])["CANTIDADORDENES0"].sum()
        pivot = pd.concat([total_consultas_prestador, pivot_1],axis=1)
        pivot.reset_index(inplace=True)
        if 1 not in pivot.columns:
            pivot[1] = 0
        if 2 not in pivot.columns:
            pivot[2] = 0
        if 3 not in pivot.columns:
            pivot[3] = 0
        if 4 not in pivot.columns:
            pivot[4] = 0

        pivot["Consultas - participación %"] =  (pivot[1]/pivot["CANTIDADORDENES0"])
        pivot["2 consultas - participación %"] =  ((pivot[2]*2)/pivot["CANTIDADORDENES0"])
        pivot["3 consultas - participación %"] =  ((pivot[3]*3)/pivot["CANTIDADORDENES0"])
        pivot["4 consultas - participación %"] =  ((pivot[4]*4)/pivot["CANTIDADORDENES0"])
        pivot["Participación %"] = 1 - pivot["Consultas - participación %"]
        pivot["Más de 4 - participación %"] = pivot["Participación %"] - (pivot["2 consultas - participación %"]+pivot["3 consultas - participación %"]+pivot["4 consultas - participación %"])

        pivot.rename(columns= {1: "Consultas - Q afiliados", 2: "2 consultas - Q afiliados", 3: "3 consultas - Q afiliados", 4: "4 consultas - Q afiliados", "desc_pre_nombre": "Razon social",
        "deno": "Especialidad","id_pre_prestador": "Prestador", "edad_afiliado": "Promedio edad de afiliados", "CANTIDADORDENES0": "Total Consultas" },inplace=True)    

        tabla = pivot[['Prestador', 'Razon social', 'Especialidad', 'Consultas - participación %', '2 consultas - participación %','3 consultas - participación %', '4 consultas - participación %', 'Más de 4 - participación %']]

        if len(tabla[tabla["Prestador"] == 218655]) ==0:
            temporal_prestador_consultas[i] = 0
            temporal_especialidad_consultas[i] = 0
        else:
            especialidad_prestador = tabla[tabla["Prestador"] == 218655].values[0][2]
            prestador = tabla[tabla["Prestador"] == 218655].iloc[:,3:]
            temporal_prestador_consultas[i] = prestador.T.values
            temporal_especialidad_consultas[i] = tabla[tabla["Especialidad"] == especialidad_prestador].iloc[:,3:].mean(axis=0).values
        
    temporal_prestador_consultas = temporal_prestador_consultas.melt(id_vars="consulta", value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
    temporal_prestador_consultas["tipo"] = "prestador"

    temporal_especialidad_consultas = temporal_especialidad_consultas.melt(id_vars="consulta",  value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
    temporal_especialidad_consultas["tipo"] = "especialidad"
    
    final_consultas = pd.concat([temporal_especialidad_consultas,temporal_prestador_consultas])
    final_consultas = final_consultas.round(2)
        
    di = {"Consultas - Q afiliados Médica": "Porcentaje de afiliados con consultas únicas", 
     "2 consultas - Q afiliados": "Porcentaje de afiliados con 2 consultas",
     "3 consultas - Q afiliado": "Porcentaje de afiliados con 3 consultas", 
     "4 consultas - Q afiliados": "Porcentaje de afiliados con 4 consultas",
     "Más de 4 - Q afiliados": "Porcentaje de afiliados con más de 4 consultas"}
    final_consultas = final_consultas.replace({"consulta": di})
    final_consultas = final_consultas.rename(columns={"value": "Participación mensual"})
    
    fig_consultas = px.bar(final_consultas, x="tipo", y="Participación mensual", barmode="stack",
             facet_col="variable",  color="consulta")

    fig_consultas.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1][:-3]))

    fig_consultas.layout.xaxis1.tickangle = -90
    fig_consultas.layout.xaxis2.tickangle = -90
    fig_consultas.layout.xaxis3.tickangle = -90
    fig_consultas.layout.xaxis4.tickangle = -90
    fig_consultas.layout.xaxis5.tickangle = -90
    fig_consultas.layout.xaxis5.tickangle = -90
    fig_consultas.layout.xaxis6.tickangle = -90
    fig_consultas.layout.xaxis7.tickangle = -90
    fig_consultas.layout.xaxis8.tickangle = -90
    fig_consultas.layout.xaxis9.tickangle = -90
    fig_consultas.layout.xaxis10.tickangle = -90
    fig_consultas.layout.xaxis11.tickangle = -90
    fig_consultas.layout.xaxis12.tickangle = -90
    fig_consultas.layout.xaxis13.tickangle =-90

    fig_consultas.layout.xaxis1.title = None
    fig_consultas.layout.xaxis2.title = None
    fig_consultas.layout.xaxis3.title = None
    fig_consultas.layout.xaxis4.title = None
    fig_consultas.layout.xaxis5.title = None
    fig_consultas.layout.xaxis5.title = None
    fig_consultas.layout.xaxis6.title = None
    fig_consultas.layout.xaxis7.title = None
    fig_consultas.layout.xaxis8.title = None
    fig_consultas.layout.xaxis9.title = None
    fig_consultas.layout.xaxis10.title = None
    fig_consultas.layout.xaxis11.title = None
    fig_consultas.layout.xaxis12.title = None
    fig_consultas.layout.xaxis13.title = None


    for i in range(0,13):
        fig_consultas.data[i]["marker"]["color"] = "#45b6fe"

    for i in range(13,26):
        fig_consultas.data[i]["marker"]["color"] = "#3792cb"

    for i in range(26,39):
        fig_consultas.data[i]["marker"]["color"] = "#296d98"

    for i in range(39,52):
        fig_consultas.data[i]["marker"]["color"] = "#1c4966"
    for i in range(52,65):
        fig_consultas.data[i]["marker"]["color"] = "black"

    fig_consultas.layout.legend = {"orientation": "h", "xanchor" :"center", "y":1.4, "x":0.5}
    fig_consultas.update_yaxes(range=[-0,1],tickformat = "%")
    
    return fig_consultas

In [174]:
calculo_graficos(prestador)

In [201]:
def calculo_graficos(prestador):
    
    prestador= int(prestador)
    #especialidad y nombre
    
    datos_prestador = querydbtopandas(f"""select top 1
       a13.id_pre_prestador,
       a13.desc_pre_nombre,
       Q.deno
    from  DBA.d_pre_prestador        a13
      JOIN  DBA.d_prof_convertir_especialidad a19  
        ON  (a13.id_pre_especialidad  = a19.id_prof_especialidad)
       LEFT JOIN dba.prestad_costo_centros E
        ON a13.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
       LEFT JOIN dba.costo_centros Q
        ON E.centro = Q.centro
    where  a13.id_pre_prestador in ({prestador})""").values[0]
    nombre = datos_prestador[1]
    especialidad = datos_prestador[2]
    especialidad = unidecode.unidecode(especialidad)

    #consultas
    menores = ["2019-01-01", "2019-02-01","2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01", "2020-01-01"]

    mayores = ["2019-02-01", "2019-03-01","2019-04-01","2019-05-01","2019-06-01","2019-07-01","2019-08-01","2019-09-01","2019-10-01","2019-11-01","2019-12-01","2020-01-01", "2020-02-01"]

    temporal_prestador_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])

    temporal_especialidad_consultas = pd.DataFrame(columns= ["consulta"], data= ['Consultas - Q afiliados Médica', '2 consultas - Q afiliados', '3 consultas - Q afiliado',
   '4 consultas - Q afiliados', 'Más de 4 - Q afiliados'])
    
    for i,j in zip(menores,mayores):
        df = querydbtopandas(f"""select a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           a11.id_afi_afiliado ,  
           Q.deno,
           a11.id_tie_dia_consumo,   
           FLOOR(DATEDIFF(year, a14.naci_fecha, a11.id_tie_dia_consumo)) edad_afiliado 

    from        DBA.ft_ordenes        a11
           join        DBA.d_pre_prestador        a13
             on        (a11.id_pre_prestador = a13.id_pre_prestador)
           join        DBA.afi_afiliados        a14
             on        (a11.id_afi_afiliado = a14.id_afiliado)
           join        DBA.d_pres_prestacion        a16
             on        (a11.id_pres_prestacion = a16.id_pres_prestacion)     
           join       DBA.d_prof_convertir_especialidad a19  
            on         (a13.id_pre_especialidad  = a19.id_prof_especialidad)
           LEFT JOIN dba.prestad_costo_centros E
            ON a11.id_pre_prestador = E.prestad and E.baja_fecha is null and e.prepaga = ( select min(PCC1.prepaga) from dba.prestad_costo_centros PCC1 where PCC1.prestad =  e.prestad )   
           LEFT JOIN dba.costo_centros Q
            ON E.centro = Q.centro
    where        (a13.id_pre_tipo in ('E', 'P')            
    and a11.id_nom_nomenclador in (5)            
    and cast(id_tie_dia_consumo as date) >= date("{i}") and cast(id_tie_dia_consumo as date) < date("{j}")
    and a13.id_afi_zona in ({zona})
    and a13.id_afi_provincia in ({provincia})
    and Q.deno like "%{especialidad}%" 
    and RTRIM(a16.prestac_pres_prestacion) not in ({prestaciones_excluidas}))
    group by a11.id_pres_prestacion,
           a16.prestac_pres_prestacion,
           a11.id_pre_prestador,
           a13.desc_pre_nombre,
           Q.deno,
           a11.id_afi_afiliado,
           a11.id_tie_dia_consumo,
           edad_afiliado""")

        total_consultas_prestador = df["id_pre_prestador"].value_counts()
        mas_de_30_consultas = total_consultas_prestador[total_consultas_prestador >= 30].index

        df = df[df["id_pre_prestador"].isin(mas_de_30_consultas)]

        reconsultas = df.groupby(["id_pre_prestador", "desc_pre_nombre", "deno", "id_afi_afiliado"])["id_afi_afiliado"].count().rename("CANTIDADORDENES0").reset_index()
        reconsultas["uno"] = 1

        pivot_1 = pd.pivot_table(reconsultas, values='uno', index=['id_pre_prestador', "desc_pre_nombre", "deno"],
                     columns=['CANTIDADORDENES0'], aggfunc=np.sum).fillna(0)

        pivot_1["Más de 4 - Q afiliados"] = pivot_1[pivot_1.columns.tolist()[4:]].sum(axis=1)

        total_consultas_prestador = reconsultas.groupby(["id_pre_prestador",  "desc_pre_nombre", "deno"])["CANTIDADORDENES0"].sum()
        pivot = pd.concat([total_consultas_prestador, pivot_1],axis=1)
        pivot.reset_index(inplace=True)
        if 1 not in pivot.columns:
            pivot[1] = 0
        if 2 not in pivot.columns:
            pivot[2] = 0
        if 3 not in pivot.columns:
            pivot[3] = 0
        if 4 not in pivot.columns:
            pivot[4] = 0

        pivot["Consultas - participación %"] =  (pivot[1]/pivot["CANTIDADORDENES0"])
        pivot["2 consultas - participación %"] =  ((pivot[2]*2)/pivot["CANTIDADORDENES0"])
        pivot["3 consultas - participación %"] =  ((pivot[3]*3)/pivot["CANTIDADORDENES0"])
        pivot["4 consultas - participación %"] =  ((pivot[4]*4)/pivot["CANTIDADORDENES0"])
        pivot["Participación %"] = 1 - pivot["Consultas - participación %"]
        pivot["Más de 4 - participación %"] = pivot["Participación %"] - (pivot["2 consultas - participación %"]+pivot["3 consultas - participación %"]+pivot["4 consultas - participación %"])

        pivot.rename(columns= {1: "Consultas - Q afiliados", 2: "2 consultas - Q afiliados", 3: "3 consultas - Q afiliados", 4: "4 consultas - Q afiliados", "desc_pre_nombre": "Razon social",
        "deno": "Especialidad","id_pre_prestador": "Prestador", "edad_afiliado": "Promedio edad de afiliados", "CANTIDADORDENES0": "Total Consultas" },inplace=True)    

        tabla = pivot[['Prestador', 'Razon social', 'Especialidad', 'Consultas - participación %', '2 consultas - participación %','3 consultas - participación %', '4 consultas - participación %', 'Más de 4 - participación %']]

        if len(tabla[tabla["Prestador"] == 218655]) ==0:
            temporal_prestador_consultas[i] = 0
            temporal_especialidad_consultas[i] = 0
        else:
            especialidad_prestador = tabla[tabla["Prestador"] == 218655].values[0][2]
            prestador = tabla[tabla["Prestador"] == 218655].iloc[:,3:]
            temporal_prestador_consultas[i] = prestador.T.values
            temporal_especialidad_consultas[i] = tabla[tabla["Especialidad"] == especialidad_prestador].iloc[:,3:].mean(axis=0).values
        
    temporal_prestador_consultas = temporal_prestador_consultas.melt(id_vars="consulta", value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
    temporal_prestador_consultas["tipo"] = "prestador"

    temporal_especialidad_consultas = temporal_especialidad_consultas.melt(id_vars="consulta",  value_vars=['2019-01-01', '2019-02-01', '2019-03-01', '2019-04-01',
       '2019-05-01', '2019-06-01', '2019-07-01', '2019-08-01', '2019-09-01',
       '2019-10-01', '2019-11-01', '2019-12-01', '2020-01-01'])
    temporal_especialidad_consultas["tipo"] = "especialidad"
    
    final_consultas = pd.concat([temporal_especialidad_consultas,temporal_prestador_consultas])
    final_consultas = final_consultas.round(2)
        
    di = {"Consultas - Q afiliados Médica": "Porcentaje de afiliados con consultas únicas", 
     "2 consultas - Q afiliados": "Porcentaje de afiliados con 2 consultas",
     "3 consultas - Q afiliado": "Porcentaje de afiliados con 3 consultas", 
     "4 consultas - Q afiliados": "Porcentaje de afiliados con 4 consultas",
     "Más de 4 - Q afiliados": "Porcentaje de afiliados con más de 4 consultas"}
    final_consultas = final_consultas.replace({"consulta": di})
    final_consultas = final_consultas.rename(columns={"value": "Participación mensual"})
    
    fig_consultas = px.bar(final_consultas, x="tipo", y="Participación mensual", barmode="stack",
             facet_col="variable",  color="consulta")

    fig_consultas.layout.legend = {"orientation": "h", "xanchor" :"center", "y":1.4, "x":0.5}
    fig_consultas.update_yaxes(range=[-0,1],tickformat = "%")
    
    return fig_consultas

In [202]:
fig_consultas = calculo_graficos(prestador)

In [203]:
fig_consultas

In [ ]:
    fig_consultas.layout.xaxis1.title = None
    fig_consultas.layout.xaxis2.title = None
    fig_consultas.layout.xaxis3.title = None
    fig_consultas.layout.xaxis4.title = None
    fig_consultas.layout.xaxis5.title = None
    fig_consultas.layout.xaxis5.title = None
    fig_consultas.layout.xaxis6.title = None
    fig_consultas.layout.xaxis7.title = None
    fig_consultas.layout.xaxis8.title = None
    fig_consultas.layout.xaxis9.title = None
    fig_consultas.layout.xaxis10.title = None
    fig_consultas.layout.xaxis11.title = None
    fig_consultas.layout.xaxis12.title = None
    fig_consultas.layout.xaxis13.title = None

In [189]:
for i in range(1,14):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13


In [199]:
var_holder = {}
 
for i in range(1,14):
    var_holder["fig_consultas.layout.xaxis" + str(i) + ".title.text"] = None

locals().update(var_holder)
 
fig_consultas.layout.xaxis9.title

layout.xaxis.Title({
    'text': 'tipo'
})

In [197]:
fig_consultas.layout.xaxis9.title

layout.xaxis.Title({
    'text': 'tipo'
})

In [200]:
fig_consultas.layout.xaxis.title.text

'tipo'

In [177]:
var_holder = {}
 
for i in range(10):
    var_holder['my_var_' + str(i)] = "iterationNumber=="+str(i)
 
locals().update(var_holder)
 
print(my_var_0)

iterationNumber==0


In [178]:
my_var_0

'iterationNumber==0'

In [ ]:
{'data': [],
'layout': {}
        }
    


In [209]:
fig_consultas.data

(Bar({
     'alignmentgroup': 'True',
     'hovertemplate': ('consulta=Porcentaje de afiliad' ... 'ón mensual=%{y}<extra></extra>'),
     'legendgroup': 'Porcentaje de afiliados con consultas únicas',
     'marker': {'color': '#636efa'},
     'name': 'Porcentaje de afiliados con consultas únicas',
     'offsetgroup': 'Porcentaje de afiliados con consultas únicas',
     'orientation': 'v',
     'showlegend': True,
     'textposition': 'auto',
     'x': array(['especialidad', 'prestador'], dtype=object),
     'xaxis': 'x',
     'y': array([0.74, 0.88]),
     'yaxis': 'y'
 }),
 Bar({
     'alignmentgroup': 'True',
     'hovertemplate': ('consulta=Porcentaje de afiliad' ... 'ón mensual=%{y}<extra></extra>'),
     'legendgroup': 'Porcentaje de afiliados con consultas únicas',
     'marker': {'color': '#636efa'},
     'name': 'Porcentaje de afiliados con consultas únicas',
     'offsetgroup': 'Porcentaje de afiliados con consultas únicas',
     'orientation': 'v',
     'showlegend': False,
 

In [212]:
final.to_csv("final_consultas.csv")

In [213]:
pd.read_csv("final_consultas.csv", index_col=0)

,consulta,variable,Participación mensual,tipo
0,Porcentaje de afiliados con consultas únicas,2019-01-01,0.74,especialidad
1,Porcentaje de afiliados con 2 consultas,2019-01-01,0.18,especialidad
2,Porcentaje de afiliados con 3 consultas,2019-01-01,0.06,especialidad
3,Porcentaje de afiliados con 4 consultas,2019-01-01,0.02,especialidad
4,Porcentaje de afiliados con más de 4 consultas,2019-01-01,0.00,especialidad
...,...,...,...,...
60,Porcentaje de afiliados con consultas únicas,2020-01-01,0.74,prestador
61,Porcentaje de afiliados con 2 consultas,2020-01-01,0.12,prestador
62,Porcentaje de afiliados con 3 consultas,2020-01-01,0.06,prestador
63,Porcentaje de afiliados con 4 consultas,2020-01-01,0.08,prestador
